# Contest with risk averse bidders and non-monetary effort

Here we use a simple example where contestants are risk averse with respect to prize money. Put differently, the expected utility of a contestant with cost parameter $c_i$ when exerting effort $x_i$ is $p_1(x_i) u(V_1)+p_2(x_i) u(V_2)-c_i x_i$ where $p_j(x_i)$ is the probability of winning $j$'th prize when exertin effort $x_i$ (given the equilibrium strategies of the other contestants) and $u$ is a strictly increasing and concave Bernoulli utility function. The contestant is risk averse with respect to prize money because $u$ is concave.

In this model, the costs of effort are non-monetary and certain. The only uncertain part is which prize (if any) the contestant wins. (If the cost of effort was monetary, we would have to use the utility function $p_1(x_i) u(V_1-c_i x_i)+p_2(x_i) u(V_2-c_i x_i)$ which would complicate the analysis).

The model with non-monetary effort cost is relatively simple because the game of the contestants (for given prizes $V_1$ and $V_2$) is the same as in Moldovanu and Sella: Instead of $V_1$ we simply have to use $W_1=u(V_1)$ and instead of $V_2$ we use $W_2=u(V_2)$. The same analysis as in the paper gives the equilibrium strategies for the contestants $b(c) = A(c)W_1+B(c)W_2$ where $A(c)$ and $B(c)$ are as in the paper. The only small difference is in the designer's problem: The constraint that the total prize money equals 1 states $V_1+V_2=1$ which in terms of $W_j$ reads $u^{-1}(W_1)+u^{-1}(W_2)=1$. Hence, the constraint when optimizing over $W_1$ and $W_2$ is $W_2=u(1-u^{-1}(W_1))$.

The code below computes the optimal first and second prizes if cost parameters are iid uniformly distributed between 0.5 and 1 and there are 3 contestants. Here we get an interior solution: the first prize is $\approx 2/3$ and the second prize $\approx 1/3$. The intuition is that giving more prize money to the good types (via a higher first prize) does not give a lot of additional utility to them (because $u$ is concave, i.e. the marginal utility is decreasing). However, the marginal utility of the second prize is rather high (with the square root utility below it is actually infinite at 0) and can therefore motivate contestants (in particular those with medium and high cost parameters) considerably.

In [ ]:
#load plotting package
using PyPlot
#load optimization package
using Optim

#########Setting up the primitives of the model#############

#lower bound type distribution
const m = 0.5

#number of contestants
const k = 3

#cdf of type distribution; here uniform
function F(a::Float64)
    if m<= a<= 1.0
        return (a-m)/(1.0-m)
    elseif a<m
        return 0.0
    else 
        return 1.0
    end
end

#pdf of type distribution; here uniform
function f(a::Float64)
    if m<= a<= 1.0
        return 1.0/(1.0-m)
    else 
        return 0.0
    end
end    

#Bernoulli utility of prize money; here √x
function u(x::Float64)
    return sqrt(x)
end

#inverse of the Bernoulli utility function of prize money; here x^2
function uinv(v::Float64)
    return v^2
end

############Setting up the equilibrium strategies of the contestants###########
A(c) = (k-1)*quadgk(a-> (1-F(a))^(k-2) * f(a)/a,c,1)[1]
B(c) = (k-1)*quadgk(a-> (1-F(a))^(k-3) * ((k-1)*F(a)-1) * f(a)/a,c,1)[1]

b(W1,W2,c) = A(c)*W1 + B(c)*W2

####### Designer's problem ########
#this function returns the expected effort of a participant multiplied with -1 (as the optimization procedure below searches for the minimum)
function minusobjective(W1::Float64)
    function wrapped_objective(c::Float64)
        return b(W1,u(1.0-uinv(W1)),c)*f(c) #note that W2 = u(1.0-uinv(W1)) by the constraint V1+V2=1
    end
    return (-1.0)*quadgk(wrapped_objective,m,1)[1]
end

#result to optimization problem (minimizes "minusobjective" with W1 between u(0.5) and u(1.0); note that V1 cannot be less than 0.5)
r = optimize(minusobjective,u(0.5),u(1.0))

println(" ")
println("Optimal first prize is ", uinv(r.minimum))
println("Optimal second prize is ", 1.0-uinv(r.minimum))

#for plotting: returns average effort of a contestant if first prize is V1
function plotobjective(V1::Float64)
    return (-1.0)*minusobjective(u(V1))
end
function plotobjective(V1::FloatRange{Float64})
    return [(-1.0)*minusobjective(u(v)) for v in V1]
end

fig, ax = subplots()
ax[:set_xlabel]("first prize")
ax[:set_ylabel]("expected effort")
ax[:plot](0.0:0.01:1.0,plotobjective(0.0:0.01:1.0),"b-",linewidth=2,alpha=0.5)


# Contests with risk averse contestants and monetary effort

In [3]:
#load plotting package
using PyPlot
#load optimization package
using Optim
#interpolation package used for solving differential equation
using Interpolations

#########Setting up the primitives of the model#############

#lower bound type distribution
const m = 0.5

#number of contestants
const k = 3

#cdf of type distribution; here uniform
function F(a::Float64)
    if m<= a<= 1.0
        return (a-m)/(1.0-m)
    elseif a<m
        return 0.0
    else 
        return 1.0
    end
end

#pdf of type distribution; here uniform
function f(a::Float64)
    if m<= a<= 1.0
        return 1.0/(1.0-m)
    else 
        return 0.0
    end
end    

#degree of risk aversion
const eta = 1.0

#Bernoulli utility of prize money; here CARA
function u(x::Float64)
    return - e^(- eta*x)
end

#Derivative of Bernoulli utility 
function up(x::Float64)
    return eta*e^(-eta*x)
end

#inverse of the Bernoulli utility function of prize money; here x^2
function uinv(v::Float64)
    return -log(-v)/eta
end


###symmetric equilibrium strategies of the contestants####

#returns the foc solved for y'
function foc_yp(x::Float64,y::Float64,V1::Float64,V2::Float64)
    numerator = up(V1-y*x)*y*(1-F(y))^(k-1)+up(V2-y*x)*y*(k-1)*(1-F(y))^(k-2)*F(y)+up(-y*x)*(1-(1-F(y))^(k-1)-(k-1)*(1-F(y))^(k-2)*F(y))
    denominator = -(u(V1-x*y)-u(-x*y))*(k-1)*(1-F(y)^(k-2)*f(y))-(u(V2-x*y)-u(-x*y))*(k-1)*(k-2)*(1-F(y))^(k-3)*F(y)*f(y)+(u(V2-x*y)-u(-x*y))*(k-1)*(1-F(y))^(k-2)*f(y)
    return numerator/denominator
end

#given intial condition b(1.0)=0, this uses the foc to solve for the equilibrium bidding function
function trace_out_y(V1::Float64,V2::Float64,epsilon::Float64)
    y = [1.0, 1.0] #starting values: type 1.0 exerts zero effort
    x = [0.0, 0.0]
    while y[end-1]>=y[end]>m && x[end]<V1/m #checks monotonicty, that types are in [m,1] and that effort cannot be higher than possible prizes
        push!(y,y[end]+epsilon*foc_yp(x[end],y[end],V1,V2))
        push!(x,x[end]+epsilon)
    end
    #check monotonicity
    if y[end]>m+0.01
        println("a monotone equilibrium was not attained at V1 = ",V1," and V2 = ",V2)
    else
        return interpolate((reverse(y),),reverse(x),Gridded(Linear()))
    end
end

test = trace_out_y(0.9,0.1,0.01)

###designer's problem###
#this function returns the expected effort of a participant multiplied with -1 (as the optimization procedure below searches for the minimum)
function minusobjective(V1::Float64)
    bidding = trace_out_y(V1,1.0-V1,0.001)
    function wrapped_objective(c::Float64)
        return bidding[c]*f(c) 
    end
    return (-1.0)*quadgk(wrapped_objective,m,1)[1]
end

#result to optimization problem (minimizes "minusobjective" with W1 between u(0.5) and u(1.0); note that V1 cannot be less than 0.5)
r = optimize(minusobjective,0.5,1.0)

println(" ")
println("Optimal first prize is ", r.minimum)
println("Optimal second prize is ", 1.0-r.minimum)

#for plotting: returns average effort of a contestant if first prize is V1
function plotobjective(V1::Float64)
    return (-1.0)*minusobjective(V1)
end
function plotobjective(V1::FloatRange{Float64})
    return [(-1.0)*minusobjective(v) for v in V1]
end

fig, ax = subplots()
ax[:set_xlabel]("first prize")
ax[:set_ylabel]("expected effort")
ax[:plot](0.0:0.01:1.0,plotobjective(0.0:0.01:1.0),"b-",linewidth=2,alpha=0.5)



a monotone equilibrium was not attained at V1 = 0.9 and V2 = 0.1
a monotone equilibrium was not attained at V1 = 0.

LoadError: LoadError: MethodError: `getindex` has no method matching getindex(::Void, ::Float64)
while loading In[3], in expression starting on line 93

6909830056250525 and V2 = 0.30901699437494745


In [1]:
using Roots

function binv(W1::Float64,W2::Float64,bid::Float64)
    function wrapper(c::Float64)
        return b(W1,W2,c)-bid
    end
    fzero(wrapper,m,1)
end

function minus_agent_obj(W1::Float64,W2::Float64,c::Float64,x::Float64)
    return (-1.0)*(W1*(1-F(binv(W1,W2,x)))^(k-1)+(k-1)*W2*F(binv(W1,W2,x))*(1-F(binv(W1,W2,x)))^(k-2)-c*x)
end

function optimal_x(W1::Float64,W2::Float64,c::Float64)
    function wrapper(x::Float64)
        return minus_agent_obj(W1,W2,c,x)
    end
    r = optimize(wrapper,0.0,b(W1,W2,m))
    return r.minimum
end

println("for c= 0.95, optimal bid is ",b(1.0,0.0,0.75),"  best response bid is ",optimal_x(1.0,0.0,0.75))

function plotbinv(x)
    return binv(1.0,0.0,x)
end
function plotb(C)
    return [b(1.0,0.0,c) for c in C]
end
plot(m:0.01:1.0,plotb(m:0.01:1.0))


LoadError: LoadError: UndefVarError: b not defined
while loading In[1], in expression starting on line 22

In [4]:
e

e = 2.7182818284590...